In [3]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, select, Float


In [4]:
# Configurando Dados do driver do navegador utilizado para raspagem de dados (Google Chrome).

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(3)
driver.get('https://www.fundsexplorer.com.br/ranking')

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 13.3MB/s]


In [5]:
fundsTable = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, '[data-element="table-ranking-container"]')
    )
)

In [6]:
data = []

for row in fundsTable.find_elements(By.TAG_NAME, 'tr'):
    registers = [register.text for register in row.find_elements(By.TAG_NAME, 'td')]

    if registers:
        data.append(registers)

In [9]:
# Criando DataFrame com os dados raspados.

df = pd.DataFrame(
    data,
    columns=[
        'Code', 
        'Sector', 
        'Current Price', 
        'Daily Liquidity', 
        'P-VP', 
        'Last Dividend',
        'Dividend Yield', 
        'DY Accumulated (3M)', 
        'DY Accumulated (6M)', 
        'DY Accumulated (12M)',
        'DY Average (3M)', 
        'DY Average (6M)', 
        'DY Average (12M)', 
        'DY Year', 
        'Price Variation',    
        'R Period', 
        'R Accumulated', 
        'Net worth', 
        'VPA', 
        'P-VPA',    
        'DY Equity', 
        'Equity Variation', 
        'RP Period', 
        'RP Accumulated',    
        'Physical Vacancy', 
        'Financial Vacancy', 
        'Q Active'
    ]
)

In [10]:
# Convertendo os dados para o tipo correto:
for column in df.columns:
    df[column] = df[column].apply(lambda item: pd.NA if (item == 'N/A') else item)

# Tratando dados capturados

- # Descarte de Registros Duplicados.
- - # Principais motivos para o descarte de dados duplicados:

- - - Precisão: Dados duplicados podem levar a uma supervalorização da importância de um determinado valor ou observação.
- - - Eficiência: Quando há muitos dados duplicados, a manipulação desses dados tende a se torna mais lenta e a consumir mais recursos computacionais do que o necessário.
- - - Consistência: Dados duplicados podem levar a inconsistências nos resultados finais e diminui a garantia que as informações estejam corretas e coerentes.

In [11]:
processedData = df.drop_duplicates(keep=False).copy()

- # Descarte de Atributos Definidos Como Irrelevantes.
- - # Atributos Descartados:

- - - (DY Accumulated (3M)): Devido ao fato de ser um atributo que não será utilizado.
- - - (DY Accumulated (6M)): Devido ao fato de ser um atributo que não será utilizado.
- - - (DY Accumulated (12M)): Devido ao fato de ser um atributo que não será utilizado.
- - - (DY Average (3M)): Devido ao fato de ser um atributo que não será utilizado.
- - - (DY Average (6M)): Devido ao fato de ser um atributo que não será utilizado.
- - - (DY Average (12M)): Devido ao fato de ser um atributo que não será utilizado.
- - - (R Period): Devido ao fato de ser um atributo que não será utilizado.
- - - (R Accumulated): Devido ao fato de ser um atributo que não será utilizado.
- - - (VPA): Devido ao fato de ser um atributo que não será utilizado.
- - - (P-VPA): Devido ao fato de ser um atributo que não será utilizado.
- - - (DY Equity): Devido ao fato de ser um atributo que não será utilizado.
- - - (RP Period): Devido ao fato de ser um atributo que não será utilizado.
- - - (RP Accumulated): Devido ao fato de ser um atributo que não será utilizado.
- - - (Net worth): Devido ao fato de ser um atributo que não será utilizado.
- - - (Physical Vacancy): Devido ao fato de que mais de 60% do seus registros estão nulos, optamos pelo descarte do atributo.
- - - (Financial Vacancy): Devido ao fato de que mais de 90% do seus registros estão nulos, optamos pelo descarte do atributo.
- - - (Q Active): Devido ao fato de ser um atributo que não será utilizado.

&nbsp;

- Devido ao fato de que alguns valores estão vazios ou não são uteis para o processamento, decidimos pelo descarte dos mesmos.

In [12]:
# Descartando atributos irrelevantes:
processedData.drop(
    [
        'DY Accumulated (3M)',
        'DY Accumulated (6M)',
        'DY Accumulated (12M)',
        'DY Average (3M)',
        'DY Average (6M)',
        'DY Average (12M)',
        'R Period',
        'R Accumulated',
        'VPA',
        'P-VPA',
        'DY Equity',
        'RP Period',
        'RP Accumulated',
        'Net worth',
        'Physical Vacancy',
        'Financial Vacancy',
        'Q Active',
    ],
    inplace=True,
    axis=1,
)

- # Convertendo Registros de Atributos.

- - # Atributos Convertidos Para Float:
- - - (Current Price)
- - - (Daily Liquidity)
- - - (Last Dividend)
- - - (Dividend Yield)
- - - (DY Year)
- - - (Price Variation)
- - - (P-VP)

&nbsp;

- Devido ao fato de que os valores foram retirados da internet, a sua maioria encontra-se no formato de String/Objeto.

In [13]:
# Convertendo Registros de Dados Para Tipo (Float).

for column in processedData.columns[2:]:
    processedData[column] = processedData[column].apply(
        lambda item: item
        if pd.isna(item) or isinstance(item, float)
        else float(
            item.replace('R$ ', '').replace('%', '').replace('.', '').replace(',', '.')
        )
        if isinstance(item, str)
        else item
    )

- # Preenchimento de Dados Ausentes.

- - # Atributos Categóricos:
- - - Para os atributos categóricos que se encontram vazios sera utilizado o termo '`INDEFINIDO`'.
- - - Utilizamos o termo '`INDEFINIDO`' apenas como um informe de que o registro do campo não foi informado. 

&nbsp;

- - # Atributos Quantitativos:
- - - Para os atributos quantitativos que se encontram vazios sera utilizado a `mediana` da coluna do atributo atual.
- - - Utilizamos a `mediana` para manter a distribuição original do atributo, assim ele não terá sua a forma de distribuição alterada.

&nbsp;

- Todos os atributos iram passar pelo prenchimento para garantir que não existam valores vazios no DataFrame.

In [14]:
# Substituindo Valores de Dados Ausentes.

# Trantando Dados Categóricos.
for column in processedData.columns[:2]:
    processedData[column].fillna("INDEFINIDO", inplace=True)

In [15]:
# Substituindo Valores de Dados Ausentes.

# Trantando Dados Quantitativos.
for column in processedData.columns[2:]:
    processedData[column].fillna(processedData[column].median(), inplace=True)

- # Identificando e Tratando Atributos com Registros Outliers.
- - # Desvio Padrão:
- - - Será utilizado o método do `Desvio Padrão` para análise de `outliers`: Esse método identifica outliers como valores que estão a uma distancia maior que 3 vezes o desvio padrão da media.
- - - Optamos pelo uso do `Desvio Padrão` por ser um dos metodos mais comuns e eficientes para esse tipo de base de dados.

&nbsp;

- - # Substituição Dos Valores Discrepantes Pelo Valor da Mediana do Atributo.
- - O método utilizado para o tratamento de outliers é o de `Substituição Dos Valores Discrepantes Pelo Valor da Mediana do Atributo`.
- - Utilizamos esse método póis ao substituir os valores outliers pela mediana, a distribuição dos dados é menos afetada pelos valores extremos, preservando assim a estrutura da distribuição original.

In [16]:
# Substituindo Registros Discrepantes dos Dados.

# Percorrendo Registros Quantitativos.
for column in processedData.columns[2:]:
    median = processedData[column].median()

    q1 = processedData[column].quantile(0.25)
    q3 = processedData[column].quantile(0.75)
    iqr = q3 - q1

    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    processedData[column] = processedData[column].apply(
        lambda item: 0.0 if item < lower else median if item > upper else item
    )

### Salvando os dados no banco de dados

In [17]:
engine = create_engine('sqlite:///../database.db', echo=False)
metadata = MetaData()
conn = engine.connect()

In [19]:
fundsTable = Table(
    'Funds',
    MetaData(),
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('Code', String(50)),
    Column('Sector', String(50)),
    Column('Current Price', Float),
    Column('Daily Liquidity', Float),
    Column('P-VP', Float),
    Column('Last Dividend', Float),
    Column('Dividend Yield', Float),
    Column('DY Year', Float),
    Column('Price Variation', Float),
    Column('Equity Variation', Float),
)

metadata.create_all(engine)

In [21]:
processedData['id'] = processedData.reset_index().index + 1

processedData.to_sql('Funds', conn, if_exists='replace', index=False)


379